In [1]:
from time import sleep
from furhat_remote_api import FurhatRemoteAPI
import speech_recognition as sr
import cv2
import opencv_jupyter_ui as jcv2
from feat import Detector
from feat.utils import FEAT_EMOTION_COLUMNS

# Detector choice
detector = Detector(device="cuda")

In [3]:
# Furhat IP address
FURHAT_IP = "127.0.1.1"

# Connect to Furhat
furhat = FurhatRemoteAPI(FURHAT_IP)
furhat.set_led(red=100, green=50, blue=50)

# Furhat faces and voices
FACES = {'TheJoker': 'Brooklyn'}
VOICES_EN = {'TheJoker': 'GregoryNeural'}

# Furhat speech
def bsay(line):
    furhat.say(text=line, blocking=True)
    sleep(1) 

# Speech recognition setup
recognizer = sr.Recognizer()

# Function to recognize speech
def recognize_speech():
    with sr.Microphone() as source:
        print("Say something!")
        audio = recognizer.listen(source, timeout=5)
    try:
        text = recognizer.recognize_google(audio)
        print("You said:", text)
        return text
    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio.")
        return None

# Function for emotion detection
def detect_emotion(frame):
    faces = detector.detect_faces(frame)
    landmarks = detector.detect_landmarks(frame, faces)
    emotions = detector.detect_emotions(frame, faces, landmarks)

    faces = faces[0]
    emotions = emotions[0]

    strongest_emotion = emotions.argmax(axis=1)
    

    for (face, top_emo) in zip(faces, strongest_emotion):
        (x0, y0, x1, y1, p) = face
        cv2.rectangle(frame, (int(x0), int(y0)), (int(x1), int(y1)), (255, 0, 0), 3)
        cv2.putText(frame, FEAT_EMOTION_COLUMNS[top_emo], (int(x0), int(y0 - 10)),
                    cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)

        # Return the detected emotion
        return FEAT_EMOTION_COLUMNS[top_emo] 


# Function to react to speech
def speech(text):
    if text:
        if "water" in text.lower():
            bsay("very tasty")
            furhat.gesture(name='Surprise')

# Function to react to emotion
def emotion(detected_emotion):
            print(f"Detected emotion: {detected_emotion}")
            if detected_emotion == "happiness":
                bsay("You are happy")
                furhat.gesture(name="Smile")
            elif detected_emotion == "sadness":
                bsay("You are sad")
                furhat.gesture(name='Oh')
            elif detected_emotion == "neutral":
                bsay("You are neutral")
                furhat.gesture(name='Wink')

# Interaction function
def interaction():
    furhat.set_face(character=FACES['TheJoker'], mask="Adult")
    furhat.set_voice(name=VOICES_EN['TheJoker'])
    bsay("Hi")
    furhat.gesture(name='ExpressDisgust')
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error: failed to capture image")
            break

        detected_emotion = detect_emotion(frame)
        speech(recognize_speech())
        emotion(detected_emotion)

        jcv2.imshow("Emotion Detection", frame)
        key = jcv2.waitKey(33) & 0xFF
        if key == 27:  # ESC pressed
            break

# Set up camera
cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# main function
if __name__ == '__main__':
    interaction()

cam.release()
jcv2.destroyAllWindows()

C:\Users\gabbe\AppData\Roaming\Python\Python311\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Detected emotion: neutral


Detected emotion: neutral
Detected emotion: neutral
Detected emotion: neutral
Detected emotion: neutral
Detected emotion: neutral
Detected emotion: neutral
Detected emotion: neutral
Detected emotion: neutral
Detected emotion: neutral
Exection is stopped
